In [1]:
#import neccessary libraries
import numpy as np
import evaluation as ev
import csv
import pandas as pd

In [2]:
#set root file name
root_file = 'Gamma-E300-n1000-part6'

In [3]:
#read cluster distribution csv as a dataframe and get noise level that minimizes cluster number
df = pd.read_csv('cluster-noise-distribution_'+root_file+'.csv')
noise = df['noise'][df['n_clusters_mean'].idxmin()]

#set run parameters
n_events = 1000
#event_n = 0

In [4]:
#modifiedtest_single_photon function (taken from test_stats.py) that returns relevant info for each particle, as
#seen in the plot output
#(x, y, z, t, e, clusterindex, pdgid, E_bound, sum(E_hit)
#this function calls our model and tests it using the root file defined in datasets.py in the single_photon_dataset function
def particle_info(n_events, noise):
    tbeta = .2
    td = .5
    nmax = n_events
    yielder = ev.TestYielderSinglePhoton()
    yielder.model.signal_threshold = noise
    event_n = []
    event_xhit = []
    event_yhit = []
    event_zhit = []
    event_Ehit = []
    event_thit = []
    event_clusters = []
    for i, (event, prediction, clustering) in enumerate(yielder.iter_clustering(tbeta, td, nmax=nmax)):
        event_n = np.append(event_n, np.full(len(event.xhit), i))
        event_xhit = np.append(event_xhit, event.xhit)
        event_yhit = np.append(event_yhit, event.yhit)
        event_zhit = np.append(event_zhit, event.zhit)
        event_Ehit = np.append(event_Ehit, event.energy)
        event_thit = np.append(event_thit, event.time)
        event_clusters = np.append(event_clusters, clustering)
    return event_n, event_xhit, event_yhit, event_zhit, event_Ehit, event_thit, event_clusters

In [5]:
#get 5 vectors for all particles of a single event
p_5vector = particle_info(n_events, noise)

100%|███████████████████████████████████████| 1000/1000 [11:18<00:00,  1.47it/s]


In [6]:
#write the 5vector as a pandas dataframe
df_5v = pd.DataFrame({'event_n': p_5vector[0].astype(np.int64), 'x': np.round(p_5vector[1].astype(np.float64), 2), 
                      'y': np.round(p_5vector[2].astype(np.float64), 2), 'z': np.round(p_5vector[3].astype(np.float64), 2), 
                      'e': np.round(p_5vector[4].astype(np.float64), 3), 't': p_5vector[5].astype(np.float64), 
                      'clusterindex': p_5vector[6].astype(np.int64)})

df_5v

,event_n,x,y,z,e,t,clusterindex
0,0,-39.27,0.40,322.07,0.005,-1.0,0
1,0,-39.27,4.40,322.07,0.004,-1.0,0
2,0,-36.49,6.01,322.07,0.004,-1.0,0
3,0,-32.33,8.41,322.07,0.004,-1.0,0
4,0,-31.64,-0.80,322.07,0.005,-1.0,0
...,...,...,...,...,...,...,...
19224003,999,-54.74,40.47,470.40,0.045,-1.0,0
19224004,999,-34.88,-56.74,470.40,0.045,-1.0,0
19224005,999,-48.45,-22.37,478.95,0.041,-1.0,0
19224006,999,-48.45,-28.37,513.15,0.041,-1.0,0


In [7]:
#write particle info dataframe to .csv 
#df_5v.to_csv('particle-info_'+root_file+'_noise-'+str(noise)+'_eventn-'+str(event_n)+'.csv', index=False)

#modifiedtest_single_photon function (taken from test_stats.py) that returns cluster per hit & energy per hit
#this function calls our model and tests it using the root file defined in datasets.py in the single_photon_dataset function
def clusters_info(n_events, noise):
    tbeta = .2
    td = .5
    nmax = n_events
    yielder = ev.TestYielderSinglePhoton()
    yielder.model.signal_threshold = noise
    event_n = []
    event_cluster = []
    cluster_energy = []
    cluster_nhits = []
    for i, (event, prediction, clustering) in enumerate(yielder.iter_clustering(tbeta, td, nmax=nmax)):
        #see https://stackoverflow.com/questions/67108215/how-to-get-sum-of-values-in-a-numpy-array-based-on-another-array-with-repetitive
        np.set_printoptions(suppress=True) #suppress scientific notation
        _, idx, _ = np.unique(clustering, return_counts=True, return_inverse=True)
        energy_values = np.round(np.bincount(idx, event.energy),3)
        
        event_cluster_unique, n_hits = np.unique(clustering, return_counts=True, return_index=False)
        event_i = np.full(len(event_cluster_unique), i, dtype=int)
        event_n = np.append(event_n, event_i)
        event_cluster = np.append(event_cluster, event_cluster_unique)
        cluster_nhits = np.append(cluster_nhits, n_hits)
        cluster_energy = np.append(cluster_energy, energy_values)
    return event_n, event_cluster, cluster_nhits, cluster_energy

#produce cluster information
ci = clusters_info(n_events, noise)

#create pandas dataframe with resuls from cluster mapping
df_ci = pd.DataFrame({'event_n': ci[0].astype(np.int64), 'clusterindex': ci[1].astype(np.int64), 'n_hits': ci[2].astype(np.int64), 
                      'energy_value': ci[3]})

In [8]:
#read file in case you dont want to produce files
n_events = 1000
df_ci = pd.read_csv('cluster-info_'+root_file+'_noise-'+str(noise)+'.csv')
df_ci = df_ci.rename(columns={'event_clusters': 'clusterindex'})

In [9]:
#group by event and sort by n_hits
df_ci = df_ci.sort_values(['event_n', 'n_hits'],ascending=[True, False])

#add column that counts the number of clusters per event starting from zero
df_ci['cluster_count'] = df_ci.groupby(df_ci['event_n'].diff().ne(0).cumsum()).cumcount()

df_ci

,event_n,clusterindex,n_hits,energy_value,cluster_count
0,0,0,17797,142.606,0
1,0,274,1359,303.029,1
2,0,1240,5,0.247,2
3,1,0,17841,144.022,0
6,1,313,1345,293.720,1
...,...,...,...,...,...
3962,998,245,2,0.040,2
3964,999,0,17865,144.225,0
3967,999,584,1305,282.502,1
3965,999,122,4,11.032,2


In [10]:
#create cluster dictionary from clusterindex and cluster count to identify the signal cluster per events
clusterN_dict = dict(zip(df_ci.clusterindex, df_ci.cluster_count))

#create cluster energy dictionary from clusterindex and energy value to correctly assign cluster energy to 
#each particle
clusterE_dict = dict(zip(df_ci.clusterindex, df_ci.energy_value))

In [11]:
#map each hit cluster using cluster dictionary
df_5v['cluster_count'] = df_5v["clusterindex"].map(clusterN_dict)
df_5v['sum(E_hit)'] = df_5v["clusterindex"].map(clusterE_dict)
df_5v = df_5v.sort_values(['event_n', 'cluster_count'],ascending=True)
df_5v

,event_n,x,y,z,e,t,clusterindex,cluster_count,sum(E_hit)
0,0,-39.27,0.40,322.07,0.005,-1.000000,0,0,144.225
1,0,-39.27,4.40,322.07,0.004,-1.000000,0,0,144.225
2,0,-36.49,6.01,322.07,0.004,-1.000000,0,0,144.225
3,0,-32.33,8.41,322.07,0.004,-1.000000,0,0,144.225
4,0,-31.64,-0.80,322.07,0.005,-1.000000,0,0,144.225
...,...,...,...,...,...,...,...,...,...
19220069,999,-29.62,69.50,322.07,0.015,-1.000000,122,2,11.032
19220082,999,-26.50,68.90,323.08,0.015,-1.000000,122,2,11.032
19220083,999,-25.46,69.50,323.08,0.010,-1.000000,122,2,11.032
19220375,999,-33.88,70.70,333.95,10.992,5.050781,122,2,11.032


In [12]:
#set cluster number to get cluster specific dataframe
cluster_n = 1

df_5v_n = df_5v[df_5v['cluster_count']== cluster_n]
df_5v_n = df_5v_n.reset_index(drop=True)
df_5v_n

,event_n,x,y,z,e,t,clusterindex,cluster_count,sum(E_hit)
0,0,-15.01,121.41,322.07,0.013,-1.000000,274,1,294.216
1,0,-16.05,112.40,323.08,0.013,-1.000000,274,1,294.216
2,0,-15.01,111.80,323.08,0.206,5.026367,274,1,294.216
3,0,-19.17,115.41,325.04,0.025,-1.000000,274,1,294.216
4,0,-17.09,113.00,325.04,0.057,5.172852,274,1,294.216
...,...,...,...,...,...,...,...,...,...
790148,999,-23.33,75.54,373.15,0.133,-1.000000,584,1,282.502
790149,999,-22.29,74.94,373.15,0.138,-1.000000,584,1,282.502
790150,999,-28.63,108.74,373.15,0.117,-1.000000,584,1,282.502
790151,999,-27.59,109.34,373.15,0.337,5.538818,584,1,282.502


In [13]:
#save dataframe to csv
df_5v_n.to_csv('particle-info_'+root_file+'_noise-'+str(noise)+'_signal'+str(cluster_n)+'.csv', index=False)